In [1]:
#typical imports
import random
from tqdm import tqdm
import numpy as np
import os

#other imports
# !pip install top2vec -q
# from top2vec import Top2Vec
# import fasttext


In [ ]:
!pip uninstall numpy -q
!pip install numpy==1.19.5 -q

In [ ]:
#train fastText

#cbow model :
ft_embeddings = fasttext.train_unsupervised('booking_sentences_processed.txt', model='cbow')

# Skipgram model :
# ft_embeddings = fasttext.train_unsupervised('fastText_training_data.txt', model='skipgram', verbose=True)


In [ ]:
ft_embeddings.save_model('fast_text_embeddings.bin')

In [2]:
#train doc2vec
!wget https://raw.githubusercontent.com/akashjorss/topic_modelling/main/booking_sentences_processed.txt
documents = []
with open('booking_sentences_processed.txt', 'r') as f:
    documents = f.readlines()
# doc2vec = Top2Vec(documents, embedding_model='doc2vec', workers=8, document_ids=list(range(0,len(documents))), speed="deep-learn", keep_documents=False)
# doc2vec.save('doc2vec_top2vec.bin')

--2021-06-10 12:38:05--  https://raw.githubusercontent.com/akashjorss/topic_modelling/main/booking_sentences_processed.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8092842 (7.7M) [text/plain]
Saving to: ‘booking_sentences_processed.txt’

booking_sentences_p 100%[===================>]   7.72M  15.4MB/s    in 0.5s    

2021-06-10 12:38:06 (15.4 MB/s) - ‘booking_sentences_processed.txt’ saved [8092842/8092842]



In [ ]:
#Build model with USE
use = Top2Vec(documents, embedding_model='universal-sentence-encoder', workers=8, document_ids=list(range(0,len(documents))), keep_documents=False)
use.save('use_top2vec.bin')

In [ ]:
#Build model with SBert
from sentence_transformers import SentenceTransformer
#download and save the sbert model
sbert_model = SentenceTransformer('stsb-mpnet-base-v2')
sbert.save("./stsb-mpnet-base-v2.pt")
#load and use SBert model
sbert = Top2Vec(documents, embedding_model_path='./stsb-mpnet-base-v2.pt', workers=8, document_ids=list(range(0,len(documents))), keep_documents=False)
sbert.save('sbert_top2vec.bin')

In [6]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [8]:
#Set up weights and biases to visualize the training of TSDAE
!pip install wandb -qqq
import wandb
wandb.login()
wandb.init(
  project='topic_modelling_booking_reviews',
  config={
      'pretrained_model':'bert-base-uncased',
      'model': 'TSDAE',
      'evaluation_metric': 'sts_processed',
      'dataset':'booking.com reviews (randomly sampled 100k)',
      'weight_decay':0,
      'scheduler':'constantlr',
      'optimizer_params':{'lr': 3e-5},
  }
)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


In [11]:
#load processed sts data for evaluation
with open('scores.txt') as f:
  scores = f.readlines()
scores = [float(s[:-1]) for s in scores]

with open('sentences1.txt') as f:
  sentences1 = f.readlines()
sentences1 = [s[:-1] for s in sentences1]

with open('sentences2.txt') as f:
  sentences2 = f.readlines()
sentences2 = [s[:-1] for s in sentences2]


In [12]:
#train TSDAE
!pip install sentence_transformers -q
from sentence_transformers import SentenceTransformer, LoggingHandler
from sentence_transformers import models, util, datasets, evaluation, losses
from torch.utils.data import DataLoader

from sentence_transformers import evaluation
evaluator = evaluation.EmbeddingSimilarityEvaluator(sentences1, sentences2, scores)

# Define your sentence transformer model using CLS pooling
model_name = 'bert-base-uncased' #can we use stsb-mpnet-base-v2?
word_embedding_model = models.Transformer(model_name)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(), pooling_mode_mean_tokens=False, pooling_mode_cls_token=True, pooling_mode_max_tokens=False)
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

model.to('cuda')

# Define a list with sentences (1k - 100k sentences)
train_sentences = documents

# Create the special denoising dataset that adds noise on-the-fly
train_dataset = datasets.DenoisingAutoEncoderDataset(train_sentences)
# train_dataset.to('cuda')

# DataLoader to batch your data
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)

# Use the denoising auto-encoder loss
train_loss = losses.DenoisingAutoEncoderLoss(model, decoder_name_or_path=model_name, tie_encoder_decoder=True)

score_list = []
steps_list = []
import matplotlib.pyplot as plt

#callback function
def call_back(score, epoch, steps):
  wandb.log({"score":score, "steps":steps, "epoch": epoch, "num_sentences":steps*8, "loss":1-score})
  print("steps:", steps, "score:", score, "loss:", 1-score)

# Call the fit method
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=1,
    weight_decay=0,
    scheduler='constantlr',
    optimizer_params={'lr': 3e-5},
    show_progress_bar=True,
    save_best_model=True,
    checkpoint_path = './tsdae_checkpoints/',
    checkpoint_save_steps = 500, 
    checkpoint_save_total_limit = 2,
    evaluation_steps=100,
    callback=call_back,
    evaluator=evaluator

)
wandb.finish()
# model.save('tsdae-model.pt')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


When tie_encoder_decoder=True, the decoder_name_or_path will be invalid.
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertLMHeadModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertLMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertLMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertLMHeadModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['bert.encoder.layer.0.crossattention.self.query.bias', 'bert.encoder.layer.6.crossattention.self.query.weight', 'bert.encoder.layer.0.crossattention.output.dense.bias

steps: 100 score: 0.4005589253567739 loss: 0.5994410746432262
steps: 200 score: 0.4338241171946967 loss: 0.5661758828053033
steps: 300 score: 0.45106368945623787 loss: 0.5489363105437621
steps: 400 score: 0.4650935183531234 loss: 0.5349064816468766
steps: 500 score: 0.47042097871936317 loss: 0.5295790212806368
steps: 600 score: 0.4688045795160717 loss: 0.5311954204839283
steps: 700 score: 0.4830351144038897 loss: 0.5169648855961103
steps: 800 score: 0.4876092581541414 loss: 0.5123907418458586
steps: 900 score: 0.49315004077243024 loss: 0.5068499592275697
steps: 1000 score: 0.4917171827937965 loss: 0.5082828172062035
steps: 1100 score: 0.4950084225548149 loss: 0.5049915774451851
steps: 1200 score: 0.49300267536951525 loss: 0.5069973246304847
steps: 1300 score: 0.4924364211803439 loss: 0.5075635788196561
steps: 1400 score: 0.48754102021489515 loss: 0.5124589797851049
steps: 1500 score: 0.4865671018465955 loss: 0.5134328981534044
steps: 1600 score: 0.4879474538463925 loss: 0.5120525461536

RuntimeError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#build top2vec model using tsdae
tsdae = Top2Vec(documents, embedding_model_path='./tsdae.pt', workers=8, document_ids=list(range(0,len(documents))), keep_documents=False)
tsdae.save('sbert_top2vec.bin')

In [ ]:
import torch
torch.save(model, "tsdae.pt")

In [ ]:
"""
@article{wang-2021-TSDAE,
    title = "TSDAE: Using Transformer-based Sequential Denoising Auto-Encoderfor Unsupervised Sentence Embedding Learning",
    author = "Wang, Kexin and Reimers, Nils and  Gurevych, Iryna", 
    journal= "arXiv preprint arXiv:2104.06979",
    month = "4",
    year = "2021",
    url = "https://arxiv.org/abs/2104.06979",
}
"""


In [ ]:
"""The performance of TSDAE reduced over time on STS possibly because of 
we did lemm., stop word removal, etc. and the sentence structure doesn't match anymore. 
The only way I can think of right now is to apply the text processing steps to STS data set and then evaluate.""" 